In [11]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [12]:
querySno = SQLQuery('snowflake')
base = querySno(r""" 
with default_flags as (
select distinct lending_business_id, business_id,loan_month,
max(case when emi_no = 1 then due_date end) over(partition by lending_business_id) as first_duedate, 
max(case when emi_no = 2 then due_date end) over(partition by lending_business_id) as second_duedate, 
max(case when emi_no = 3 then due_date end) over(partition by lending_business_id) as third_duedate,
max(case when emi_no = 4 then due_date end) over(partition by lending_business_id) as fourth_duedate,
max(case when emi_no = 5 then due_date end) over(partition by lending_business_id) as fifth_duedate,
max(case when emi_no = 6 then due_date end) over(partition by lending_business_id) as sixth_duedate,
max(case when emi_no = 1 then defalt end) over(partition by lending_business_id) as fpd, 
max(case when emi_no = 2 then defalt end) over(partition by lending_business_id) as spd, 
max(case when emi_no = 3 then defalt end) over(partition by lending_business_id) as tpd,
max(case when emi_no = 4 then defalt end) over(partition by lending_business_id) as frpd,
max(case when emi_no = 5 then defalt end) over(partition by lending_business_id) as ffpd,
max(case when emi_no = 6 then defalt end) over(partition by lending_business_id) as sxpd,
max(case when emi_no = 1 then total_emi end) over(partition by lending_business_id) as first_emi_amount,
max(case when emi_no = 2 then total_emi end) over(partition by lending_business_id) as second_emi_amount,
max(case when emi_no = 3 then total_emi end) over(partition by lending_business_id) as third_emi_amount,
max(case when emi_no = 4 then total_emi end) over(partition by lending_business_id) as fourth_emi_amount,
max(case when emi_no = 5 then total_emi end) over(partition by lending_business_id) as fifth_emi_amount,
max(case when emi_no = 6 then total_emi end) over(partition by lending_business_id) as sixth_emi_amount
from (
select a.*,json_extract_path_text(c.predict_meta, 'business_id') as b1_id, json_extract_path_text(c.predict_meta, 'data.business_id') as b2_id,
case when b1_id is null then b2_id else b1_id end as business_id,
month(a.created_at) as month_no,
case when paid_on_time = 'true' then 0
when paid_on_time = 'false' then 1 else null end as default_flag,
dense_rank() over(partition by a.lending_business_id order by due_date) as emi_no,
min(a.created_at) over (partition by a.lending_business_id) as min_created_at,
month(min_created_at) as loan_month,
max(default_flag) over(partition by a.lending_business_id, due_date) as defalt,
sum(due_amount) over(partition by a.lending_business_id, due_date) as amount_due,
sum(payment) over(partition by a.lending_business_id, due_date) as payment_done,
amount_due/100 + payment_done/100 as total_emi
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS c on a.lending_business_id = c.lending_business_id
where date(due_date) <= date('2023-06-05')
-- and a.lending_business_id = '909564ef-6b22-4303-9df5-fc4e892fef01'
)
)
, first_cure as (
select *,transaction_date as first_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) rnk
from (
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as first_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where first_amount_cured >= first_emi_amount
)
where rnk = 1
)
, second_cure as (
select *, transaction_date as second_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
t.running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as second_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where second_amount_cured >= first_emi_amount + second_emi_amount
)
where rnk = 1
)

,third_cure as (
select *, transaction_date as third_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as third_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where third_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount
)
where rnk = 1
)

,fourth_cure as (
select *, transaction_date as fourth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount ,t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as fourth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where fourth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount
)
where rnk = 1
)

,fifth_cure as (
select *, transaction_date as fifth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount,b.fifth_emi_amount ,t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate, b.fifth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as fifth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where fifth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount + fifth_emi_amount
)
where rnk = 1
)

,sixth_cure as (
select *, transaction_date as sixth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount,
b.fifth_emi_amount,b.sixth_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate, b.fifth_duedate, b.sixth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as sixth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where sixth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount + fifth_emi_amount + sixth_emi_amount
)
where rnk = 1
)

select a.lending_business_id, a.business_id, a.first_duedate,date(f.first_cure_date)as first_cure_date, a.second_duedate,date(s.second_cure_date)as second_cure_date, 
a.third_duedate,date(t.third_cure_date) as third_cure_date, a.fourth_duedate,date(fr.fourth_cure_date) as fourth_cure_date, a.fifth_duedate, date(ff.fifth_cure_date) as fifth_cure_date,
a.sixth_duedate, date(sx.sixth_cure_date) as sixth_cure_date,
fpd, spd, tpd, frpd, ffpd,sxpd,
case when fpd = 1 and first_amount_cured >= a.first_emi_amount then 1 else 0 end as first_cure_flag,
case when spd = 1 and second_amount_cured >= a.first_emi_amount + a.second_emi_amount then 1 else 0 end as second_cure_flag,
case when tpd = 1 and third_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount then 1 else 0 end as third_cure_flag,
case when frpd = 1 and fourth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount then 1 else 0 end as fourth_cure_flag,
case when ffpd = 1 and fifth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount + a.fifth_emi_amount then 1 else 0 end as fifth_cure_flag,
case when sxpd = 1 and sixth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount + a.fifth_emi_amount + a.sixth_emi_amount then 1 else 0 end as sixth_cure_flag,
case when first_cure_flag = 1 then datediff(day,a.first_duedate,f.first_cure_date) else null end as first_cure_days,
case when second_cure_flag = 1 then datediff(day,a.second_duedate,s.second_cure_date) else null end as second_cure_days,
case when third_cure_flag = 1 then datediff(day,a.third_duedate,t.third_cure_date) else null end as third_cure_days,
case when fourth_cure_flag = 1 then datediff(day,a.fourth_duedate,fr.fourth_cure_date) else null end as fourth_cure_days,
case when fifth_cure_flag = 1 then datediff(day, a.fifth_duedate, ff.fifth_cure_date) else null end as fifth_cure_days,
case when sixth_cure_flag = 1 then datediff(day, a.sixth_duedate, sx.sixth_cure_date) else null end as sixth_cure_days,
a.first_emi_amount, f.first_amount_cured, a.second_emi_amount,(s.second_amount_cured - f.first_amount_cured) as second_amt_cured, a.third_emi_amount, 
(t.third_amount_cured - second_amt_cured) as third_amt_cured, a.fourth_emi_amount, (fr.fourth_amount_cured - third_amt_cured) as fourth_amt_cured,
(ff.fifth_amount_cured - fourth_amt_cured) as fifth_amt_cured, (sx.sixth_amount_cured - fifth_amt_cured) as sixth_amt_cured
from default_flags a 
left join first_cure f on a.lending_business_id = f.lending_business_id
left join second_cure s on a.lending_business_id = s.lending_business_id
left join third_cure t on a.lending_business_id = t.lending_business_id
left join fourth_cure fr on a.lending_business_id = fr.lending_business_id
left join fifth_cure ff on a.lending_business_id = ff.lending_business_id
left join sixth_cure sx on a.lending_business_id = sx.lending_business_id
where a.first_duedate <= date('2023-06-05')
-- and a.business_id = '111eb19b-cd06-4943-b9f5-c111ae38189e'
order by first_duedate desc
""")

In [13]:
base.shape

(2835, 42)

In [14]:
base.head()

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fourth_duedate,fourth_cure_date,...,first_emi_amount,first_amount_cured,second_emi_amount,second_amt_cured,third_emi_amount,third_amt_cured,fourth_emi_amount,fourth_amt_cured,fifth_amt_cured,sixth_amt_cured
0,06ba9c27-1721-47c2-b028-eede0dac6905,a784e047-dae9-45f4-8b62-e952fa462489,2023-06-05,None,None,None,None,None,None,None,...,98.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8d3f5c69-6066-4c40-bf6d-028cb4b27968,5eabf358-2692-4532-93a4-66a411f43bcb,2023-06-05,2023-06-05,None,None,None,None,None,None,...,153.33,153.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06296573-9834-4351-b450-c59f44e9741e,c4cec416-702f-4067-ba46-84088fbc0a5f,2023-06-05,None,None,None,None,None,None,None,...,235.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8a9665eb-72b5-41a3-b660-a88bd14d9e7b,0dfc2538-123f-46fa-9b52-dae59e1433cf,2023-06-05,2023-06-05,None,None,None,None,None,None,...,271.82,271.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ae3e217c-b791-411f-a8ad-cc90da586821,fd605da3-6657-4e9b-9dca-dae55c65f944,2023-06-05,2023-06-16,None,None,None,None,None,None,...,975.33,1010.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
base['everDPD'] = base.apply(lambda x: max(x['fpd'],x['spd'],x['tpd'],x['frpd'],x['ffpd'],x['sxpd']),axis=1)

##### everDPD calculations

In [16]:
base['fpd_plus_3'] = 0
base['spd_plus_3'] = 0
base['tpd_plus_3'] = 0
base['frpd_plus_3'] = 0
base['ffpd_plus_3'] = 0
base['sxpd_plus_3'] = 0

base.loc[(base['fpd']==1) & (base['first_cure_days']>=3), 'fpd_plus_3'] = 1
base.loc[(base['spd']==1) & (base['second_cure_days']>=3), 'spd_plus_3'] = 1
base.loc[(base['tpd']==1) & (base['third_cure_days']>=3), 'tpd_plus_3'] = 1
base.loc[(base['frpd']==1) & (base['fourth_cure_days']>=3), 'frpd_plus_3'] = 1
base.loc[(base['ffpd']==1) & (base['fifth_cure_days']>=3), 'ffpd_plus_3'] = 1
base.loc[(base['sxpd']==1) & (base['sixth_cure_days']>=3), 'sxpd_plus_3'] = 1

In [17]:
base['fpd_plus_15'] = 0
base['spd_plus_15'] = 0
base['tpd_plus_15'] = 0
base['frpd_plus_15'] = 0
base['ffpd_plus_15'] = 0
base['sxpd_plus_15'] = 0

base.loc[(base['fpd']==1) & (base['first_cure_days']>=15), 'fpd_plus_15'] = 1
base.loc[(base['spd']==1) & (base['second_cure_days']>=15), 'spd_plus_15'] = 1
base.loc[(base['tpd']==1) & (base['third_cure_days']>=15), 'tpd_plus_15'] = 1
base.loc[(base['frpd']==1) & (base['fourth_cure_days']>=15), 'frpd_plus_15'] = 1
base.loc[(base['ffpd']==1) & (base['fifth_cure_days']>=15), 'ffpd_plus_15'] = 1
base.loc[(base['sxpd']==1) & (base['sixth_cure_days']>=15), 'sxpd_plus_15'] = 1

In [18]:
base['fpd_plus_30'] = 0
base['spd_plus_30'] = 0
base['tpd_plus_30'] = 0
base['frpd_plus_30'] = 0
base['ffpd_plus_30'] = 0
base['sxpd_plus_30'] = 0

base.loc[(base['fpd']==1) & (base['first_cure_days']>=30), 'fpd_plus_30'] = 1
base.loc[(base['spd']==1) & (base['second_cure_days']>=30), 'spd_plus_30'] = 1
base.loc[(base['tpd']==1) & (base['third_cure_days']>=30), 'tpd_plus_30'] = 1
base.loc[(base['frpd']==1) & (base['fourth_cure_days']>=30), 'frpd_plus_30'] = 1
base.loc[(base['ffpd']==1) & (base['fifth_cure_days']>=30), 'ffpd_plus_30'] = 1
base.loc[(base['sxpd']==1) & (base['sixth_cure_days']>=30), 'sxpd_plus_30'] = 1

##### EverDPDs

In [19]:
base['everDPD_3'] = base.apply(lambda x: max(x['fpd_plus_3'],x['spd_plus_3'],x['tpd_plus_3'],x['frpd_plus_3'],x['ffpd_plus_3'],x['sxpd_plus_3']),axis=1)
base['everDPD_15'] = base.apply(lambda x: max(x['fpd_plus_15'],x['spd_plus_15'],x['tpd_plus_15'],x['frpd_plus_15'],x['ffpd_plus_15'],x['sxpd_plus_15']),axis=1)
base['everDPD_30'] = base.apply(lambda x: max(x['fpd_plus_30'],x['spd_plus_30'],x['tpd_plus_30'],x['frpd_plus_30'],x['ffpd_plus_30'],x['sxpd_plus_30']),axis=1)

In [20]:
print("total accounts = {}".format(base.shape[0]))
print("everDPD_count = {}, everDPD% = {}".format(base['everDPD'].sum(),base['everDPD'].sum()/len(base)))
print("everDPD_3_count = {}, everDPD_3% = {}".format(base['everDPD_3'].sum(),base['everDPD_3'].sum()/len(base)))
print("everDPD_15_count = {}, everDPD_15% = {}".format(base['everDPD_15'].sum(),base['everDPD_15'].sum()/len(base)))
print("everDPD_30_count = {}, everDPD_30% = {}".format(base['everDPD_30'].sum(),base['everDPD_30'].sum()/len(base)))

total accounts = 2835
everDPD_count = 928.0, everDPD% = 0.327336860670194
everDPD_3_count = 639, everDPD_3% = 0.2253968253968254
everDPD_15_count = 463, everDPD_15% = 0.16331569664902998
everDPD_30_count = 375, everDPD_30% = 0.13227513227513227


In [21]:
cols = ['lending_business_id','fpd','spd','tpd','frpd','ffpd','sxpd','sixth_cure_days','everDPD','everDPD_15']

In [22]:
# Fully aged accounts (till 5th Jun'23)
base.loc[~base['sxpd'].isnull(),cols].shape

(166, 10)

In [23]:
base.loc[base['sxpd']==1,cols].shape

(73, 10)

In [50]:
base.to_clipboard()

In [24]:
base.head()

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fourth_duedate,fourth_cure_date,...,sxpd_plus_15,fpd_plus_30,spd_plus_30,tpd_plus_30,frpd_plus_30,ffpd_plus_30,sxpd_plus_30,everDPD_3,everDPD_15,everDPD_30
0,06ba9c27-1721-47c2-b028-eede0dac6905,a784e047-dae9-45f4-8b62-e952fa462489,2023-06-05,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,8d3f5c69-6066-4c40-bf6d-028cb4b27968,5eabf358-2692-4532-93a4-66a411f43bcb,2023-06-05,2023-06-05,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,06296573-9834-4351-b450-c59f44e9741e,c4cec416-702f-4067-ba46-84088fbc0a5f,2023-06-05,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,8a9665eb-72b5-41a3-b660-a88bd14d9e7b,0dfc2538-123f-46fa-9b52-dae59e1433cf,2023-06-05,2023-06-05,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,ae3e217c-b791-411f-a8ad-cc90da586821,fd605da3-6657-4e9b-9dca-dae55c65f944,2023-06-05,2023-06-16,None,None,None,None,None,None,...,0,0,0,0,0,0,0,1,0,0


In [25]:
querySno = SQLQuery('snowflake')
fico = querySno(r""" 
select a.lending_business_id, 
case when json_extract_path_text(predict_meta,'business_id') is null
     then json_extract_path_text(json_extract_path_text(predict_meta,'data') , 'business_id')
     else json_extract_path_text(predict_meta,'business_id') end as business_id,
b.fico_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" b
on a.LENDING_BUSINESS_ID = b.LENDING_BUSINESS_ID
and fico_score is not null
and fico_score != ''
and date(b.created_at) <= date(a.created_at)
where json_extract_path_text(a.rejection_reasons, 'data') = '[]'
or a.is_overridden = 'TRUE'
""")

In [26]:
fico.shape

(5970, 3)

In [27]:
target = base[['lending_business_id','business_id','everDPD_15']]

In [28]:
querySno = SQLQuery('snowflake')
lending_base = querySno(r""" 
with approved as (
select distinct a.lending_business_id, 
case when json_extract_path_text(predict_meta,'business_id') is null
     then json_extract_path_text(json_extract_path_text(predict_meta,'data') , 'business_id')
     else json_extract_path_text(predict_meta,'business_id') end as business_id,
a.created_at,
case when b.lending_business_id is not null then 1 else 0 end as drawn_flag
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" b 
on a.lending_business_id = b.lending_business_id
where json_extract_path_text(rejection_reasons, 'data') = '[]'
or is_overridden = 'TRUE'
)

-- select count(distinct business_id) from (
select a.*
from approved a 
join (select distinct business_id from prod_db.adhoc.lending_response_base_w_plaid_match_v2_20230623) b 
on a.business_id = b.business_id 
and a.created_at >= date('2022-11-29') and a.created_at <= date('2023-06-05')
-- )
""")

In [29]:
lending_base.shape

(888, 4)

In [30]:
lending_base.head()

,lending_business_id,business_id,created_at,drawn_flag
0,f7b79638-cabf-455f-b717-72c2392f8300,30cbab06-f644-41e1-b004-4c27abefeb06,2023-03-13 19:00:04.030422+00:00,1
1,36398b44-6c16-4f6b-98dd-49570cacc3d8,b8e7246a-685f-416b-9702-24f8cf9e412b,2023-05-12 19:01:07.839266+00:00,0
2,3e9d16fa-8a51-46b6-8122-69a62c3aaea7,42cda068-9eca-4939-8df5-637f21fb11b1,2023-01-25 18:07:44.554263+00:00,1
3,53860aa9-a644-418b-96ff-581678de4fb3,1e13884b-af74-4e26-b696-120836ae856e,2023-02-24 19:01:55.133047+00:00,1
4,4ea3d907-4df7-45c2-a44e-d271bd8a84ef,748ddb34-ca20-4616-8ed1-b0d7bed0387d,2023-01-27 15:45:15.299546+00:00,1


In [31]:
df1 = lending_base.merge(target,how='left',left_on=['lending_business_id','business_id'],right_on=['lending_business_id','business_id'])
df1.shape

(888, 5)

In [32]:
df1.head()

,lending_business_id,business_id,created_at,drawn_flag,everDPD_15
0,f7b79638-cabf-455f-b717-72c2392f8300,30cbab06-f644-41e1-b004-4c27abefeb06,2023-03-13 19:00:04.030422+00:00,1,0.0
1,36398b44-6c16-4f6b-98dd-49570cacc3d8,b8e7246a-685f-416b-9702-24f8cf9e412b,2023-05-12 19:01:07.839266+00:00,0,NaN
2,3e9d16fa-8a51-46b6-8122-69a62c3aaea7,42cda068-9eca-4939-8df5-637f21fb11b1,2023-01-25 18:07:44.554263+00:00,1,0.0
3,53860aa9-a644-418b-96ff-581678de4fb3,1e13884b-af74-4e26-b696-120836ae856e,2023-02-24 19:01:55.133047+00:00,1,0.0
4,4ea3d907-4df7-45c2-a44e-d271bd8a84ef,748ddb34-ca20-4616-8ed1-b0d7bed0387d,2023-01-27 15:45:15.299546+00:00,1,0.0


In [33]:
# drawn but not aged
df1[(df1['drawn_flag']==1) & (df1['everDPD_15'].isnull())].shape

(112, 5)

In [34]:
df = df1.merge(fico,left_on=['lending_business_id','business_id'],right_on=['lending_business_id','business_id'])
df.shape

(888, 6)

In [35]:
df['dpd_flag'] = 0
df.loc[~(df['everDPD_15'].isnull()),'dpd_flag'] = 1

In [36]:
df['dpd_flag'].sum()

483

In [37]:
df['fico_score'] = df['fico_score'].astype('int')

In [38]:
df['fico_flag'] = 0
df.loc[((df['fico_score']>600) & (df['fico_score']<=630)) | ((df['fico_score']>=720) & (df['fico_score']<=850)),'fico_flag'] = 1

In [39]:
print(df.shape)
print(df['drawn_flag'].sum())
print(df['dpd_flag'].sum())
print(df['everDPD_15'].sum())

(888, 8)
595
483
78.0


In [40]:
df_final = df[(df['dpd_flag']==1) | ((df['dpd_flag']==0) & (df['fico_flag']==1))]
df_final.shape

(708, 8)

In [41]:
df_final['target'] = 0
df_final.loc[df['dpd_flag']==1, 'target'] = df_final['everDPD_15']
df_final.loc[(df['fico_flag']==1) & (df['fico_score']<630), 'target'] = 1

/var/folders/v3/dy2p0xqd3mz5bvjtls49l7t80000gn/T/ipykernel_7453/1725682920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['target'] = 0


In [42]:
df_final.sample(10)

,lending_business_id,business_id,created_at,drawn_flag,everDPD_15,fico_score,dpd_flag,fico_flag,target
299,e5ccf18f-5598-4880-8b05-8ff925f36faf,6459dfb4-a8b0-4aa4-a6e5-25675c1828f4,2023-03-13 21:30:41.985220+00:00,1,0.0,694,1,0,0
0,f7b79638-cabf-455f-b717-72c2392f8300,30cbab06-f644-41e1-b004-4c27abefeb06,2023-03-13 19:00:04.030422+00:00,1,0.0,711,1,0,0
39,5249377a-500b-42a7-af3e-109b648c928d,b14a4191-4139-446d-aa66-b071e4eb8ccb,2023-03-24 17:29:54.080531+00:00,1,NaN,800,0,1,0
680,230fdec9-65bb-4329-8328-c6abfcc9aa5c,0ea872e1-c477-42b6-a472-b1b124512520,2023-01-10 17:20:33.623053+00:00,1,0.0,624,1,1,1
878,d68eb423-21bc-4c23-a728-46cda76c9e8a,b026d81d-7891-4429-947c-257a7e500867,2023-02-12 11:43:18.229350+00:00,0,NaN,757,0,1,0
385,52f2c87a-b723-482b-9a88-1949b1b65bf3,09b2ed60-0f0c-4df5-8238-2eecf4f38e0a,2023-04-06 07:05:11.737136+00:00,0,NaN,836,0,1,0
206,618a846f-2803-4495-9ff0-d83d0f9ab94a,4a45215c-7702-4906-a879-c0e754bf0894,2022-12-19 15:42:04.165551+00:00,0,NaN,602,0,1,1
497,126cda86-4613-4041-8d42-4de0ee161810,f7e7e6cd-0be7-44d6-9ca0-c0ecdb9e6be1,2023-02-24 16:57:41.978439+00:00,1,0.0,644,1,0,0
673,c0f0e5f0-9bf9-4570-8dbe-4acc99fec2b3,fb43f75f-05f8-499b-8f35-6ac4d9bdfff7,2023-02-11 12:01:42.647520+00:00,1,0.0,644,1,0,0
735,d8cdba53-2012-4ac4-b926-2e474ce99dba,65cc703d-1ed0-4cc5-bffd-4fa6b15d733a,2023-04-10 17:10:46.345006+00:00,0,NaN,775,0,1,0


In [43]:
print('df length = {}'.format(len(df_final)))
print('target rate = {}'.format(df_final['target'].sum()/len(df_final)))

df length = 708
target rate = 0.20903954802259886


In [44]:
df.shape

(888, 8)

In [45]:
df_final = df_final.drop(['dpd_flag','fico_flag'],axis=1)

In [46]:
# from sqlalchemy.types import NVARCHAR
# q = SQLQuery('snowflake')

# df_final.to_sql(name='lending_base_target_23jun',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in df_final})

In [10]:
df = pd.read_csv('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/base_data_target_jun22.csv')
df.shape

(2819, 42)

In [7]:
df.shape

(646, 243)

In [8]:
df.target.value_counts()

0    513
1    133
Name: target, dtype: int64

In [9]:
df.head()

,business_id,lending_business_id,decision_date,drawn_flag,everDPD_15,fico_score,target,loans_flag,payroll_flag,pos_flag,...,sum_credits_grt_1500_6M,ratio_credits_lessthan_100_1M_3M,ratio_credits_lessthan_100_1M_6M,ratio_credits_grt_500_1M_3M,ratio_credits_grt_500_1M_6M,ratio_credits_grt_1500_1M_3M,ratio_credits_grt_1500_1M_6M,txn_each_mth_flag,txn_grt_100_each_mth_flag,txn_flag
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,1,0,721,0,0.0,1.0,1.0,...,71528.95,0.333333,0.20,0.363636,0.200000,0.40,0.235294,1.0,1.0,1.0
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,1,0,846,0,1.0,0.0,1.0,...,42851.39,0.272727,0.15,0.473684,0.243243,0.25,0.125000,1.0,1.0,1.0
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,1,0,720,0,0.0,0.0,0.0,...,13647.10,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,1,0,720,0,0.0,1.0,0.0,...,79422.72,NaN,NaN,0.181818,0.105263,0.10,0.058824,1.0,1.0,1.0
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,1,0,687,0,0.0,0.0,0.0,...,1500.00,NaN,NaN,0.333333,0.166667,0.50,0.500000,0.0,0.0,1.0
